In [1]:
from bs4 import BeautifulSoup as bs
from time import sleep
import pandas as pd
import requests
import json
import gc

## Вариант 1
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    - Наименование вакансии.
    - Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
    - Ссылку на саму вакансию.
    - Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [4]:
class HH_Parser():
    """Интерфейс для парсинга вакансий на сайте hh.ru в Москве"""
    def __init__(self, headers:dict=None):
        self.url = 'https://hh.ru/search/vacancy'
        self.headers={'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0'}


    def get_vacancy(self, search_query:str, first_n_pages=0):
        self.result = []
        self.params={
            'text': search_query,
            'salary':'',
            'area':1,
            'ored_clusters':'true',
            'enable_snippets':'true',
            'page':0
            }

        def get_vacancies_per_one_page(self, page_number):
            self.params['page'] = page_number
            dom = self._get_page_dom(self.params)
            vacancies_on_current_page = self._get_vacancies_on_current_page(dom)
            return vacancies_on_current_page

        number_of_pages = self._get_last_page_number()
        if number_of_pages < first_n_pages:
            print(f'Предупреждение: На странице всего {number_of_pages} страниц')
            first_n_pages = number_of_pages

        if first_n_pages == 0:
            n_pages_to_parse = number_of_pages
        else:
            n_pages_to_parse = first_n_pages-1

        for i in range(n_pages_to_parse+1):
            vacancies_on_current_page = get_vacancies_per_one_page(self, i)
            sleep(2)
            self.result.extend(vacancies_on_current_page)
        return self.result




    def _get_vacancies_on_current_page(self, dom):
        page_result = []
        all_vacancies_info = dom.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})

        for vacancy_info in all_vacancies_info:
            minimum_salary, maximum_salary, currency = None, None, None
            try:
                salary = vacancy_info.find_all('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})[0].text
                if salary:
                    minimum_salary, maximum_salary, currency = self._agregate_salary(salary)
            except:
                pass
            
            vacancy_url = vacancy_info.find_all('a', {'class': 'serp-item__title'})[0]['href']
            vacancy_title = vacancy_info.find_all('a', {'class': 'serp-item__title'})[0].text
            vacancy_dict = {
                'title': vacancy_title,
                'url': vacancy_url,
                'minimum_salary': minimum_salary,
                'maximum_salary': maximum_salary,
                'salary_currency': currency
            }
            page_result.append(vacancy_dict)
        return page_result

            


    def _get_last_page_number(self):
        dom = self._get_page_dom(self.params)
        page_button_containers = dom.find_all('a', {'data-qa': 'pager-page'})
        last_page_number = page_button_containers[-1].find_all('span')[0].text
        last_page_number = int(last_page_number) - 1
        return last_page_number


    def _get_page_dom(self, params):
        response = requests.get(self.url, headers=self.headers, params=params)
        if response.status_code == 200:
            dom = bs(response.text, 'html.parser')
            return dom

        else: 
            print('<self._get_page_dom> Bad response, code:', response.status_code)


    def _agregate_salary(self, salary:str):
        if 'руб' in salary:
            currency = 'RUB'
        elif 'USD' in salary:
            currency = 'USD'
        else:
            currency = 'Unknown'

        nums = []
        salary_split_list = salary.split(' ')
        for salary_item in salary_split_list:
            if not 'от' in salary_item and not 'до' in salary_item and not 'руб' in salary_item and not 'USD' in salary_item:
                nums.append(salary_item)

        minimum_salary = 0
        maximum_salary = 0

        if len(nums)==2:
            minimum_salary = nums[0]
            maximum_salary = nums[1]

        elif len(nums)==1 and 'от' in salary.lower():
            minimum_salary = nums[0]

        elif len(nums)==1 and 'до' in salary.lower():
            maximum_salary = nums[0]

        elif len(nums)==1:
            minimum_salary = nums[0]
        
        return minimum_salary, maximum_salary, currency



In [5]:
parser = HH_Parser()

In [6]:
python = parser.get_vacancy('python')
len(python), python[:3]

In [195]:
with open('python_vacancies.json', 'w') as f:
    json.dump(fp=f, obj=python)

In [ ]:
del python
gc.collect()

In [8]:
data_scientist = parser.get_vacancy('data science')
len(data_scientist), data_scientist[:3]

(60,
 [{'title': 'Data scientist',
   'url': 'https://nizhny-tagil.hh.ru/vacancy/74376536?from=vacancy_search_list&query=data+science',
   'minimum_salary': 0,
   'maximum_salary': 0,
   'salary_currency': 'RUB'},
  {'title': 'Аналитик данных (Data Scientist) / ML-разработчик',
   'url': 'https://nizhny-tagil.hh.ru/vacancy/77223284?from=vacancy_search_list&query=data+science',
   'minimum_salary': None,
   'maximum_salary': None,
   'salary_currency': None},
  {'title': 'Data Scientist / ML Инженер',
   'url': 'https://nizhny-tagil.hh.ru/vacancy/76827021?from=vacancy_search_list&query=data+science',
   'minimum_salary': None,
   'maximum_salary': None,
   'salary_currency': None}])

In [199]:
with open('ds_vacancies.json', 'w') as f:
    json.dump(fp=f, obj=data_scientist)

### Lesson 4 homework

1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).

In [9]:
from pymongo import MongoClient

In [10]:
database = MongoClient()

In [11]:
database

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [12]:
database.list_database_names()

['admin', 'config', 'local']

In [14]:
db = database.hh_db

In [23]:
db.ds_vacancies.insert_one(data_scientist[0])

In [24]:
for  i in db.ds_vacancies.find():
    print(i)

{'_id': ObjectId('63fa498a96f623e4c9da9b10'), 'title': 'Data scientist', 'url': 'https://nizhny-tagil.hh.ru/vacancy/74376536?from=vacancy_search_list&query=data+science', 'minimum_salary': 0, 'maximum_salary': 0, 'salary_currency': 'RUB'}


In [25]:
db.ds_vacancies.delete_many({})

In [26]:
for vacancy in data_scientist:
    db.ds_vacancies.insert_one(vacancy)


In [27]:
a = db.ds_vacancies.find()
for i in range(3):
    print(a.next())

{'_id': ObjectId('63fa498a96f623e4c9da9b10'), 'title': 'Data scientist', 'url': 'https://nizhny-tagil.hh.ru/vacancy/74376536?from=vacancy_search_list&query=data+science', 'minimum_salary': 0, 'maximum_salary': 0, 'salary_currency': 'RUB'}
{'_id': ObjectId('63fa4a4a96f623e4c9da9b11'), 'title': 'Аналитик данных (Data Scientist) / ML-разработчик', 'url': 'https://nizhny-tagil.hh.ru/vacancy/77223284?from=vacancy_search_list&query=data+science', 'minimum_salary': None, 'maximum_salary': None, 'salary_currency': None}
{'_id': ObjectId('63fa4a4a96f623e4c9da9b12'), 'title': 'Data Scientist / ML Инженер', 'url': 'https://nizhny-tagil.hh.ru/vacancy/76827021?from=vacancy_search_list&query=data+science', 'minimum_salary': None, 'maximum_salary': None, 'salary_currency': None}


In [28]:
type(a.next())

dict

In [49]:
def renew_data_science_vacancies():
    # getting the new data
    parser = HH_Parser()
    vacancies = parser.get_vacancy('data science', first_n_pages=3)
    print(len(vacancies))
    counter = 0
    deleted = 0

    # clearing the db from unactive vacancies
    for db_vacancy in db.ds_vacancies.find():
        is_the_vacancy_in_database = False
        for vacancy in vacancies:
            if db_vacancy['url'] == vacancy['url']:
                is_the_vacancy_in_database_active = True
        if not is_the_vacancy_in_database_active:
            db.ds_vacancies.delete_one({'_id': db_vacancy['_id']})
            deleted += 1

    # adding the new vacancies to the database
    for vacancy in vacancies:
        is_the_vacancy_in_database = False
        for db_vacancy in db.ds_vacancies.find():
            if db_vacancy['url'] == vacancy['url']:
                is_the_vacancy_in_database = True
        if not is_the_vacancy_in_database:
            db.ds_vacancies.insert_one(vacancy)
            counter += 1
    print(f'Готово, добавлено {counter} вакансий' if counter != 0 else 'Ни одной вакансии не добавлено')
    print(f'Готово, удалено {deleted} вакансий' if deleted != 0 else 'Ни одной вакансии не удалено')


        

In [52]:
db.ds_vacancies.delete_many({})

In [53]:
renew_data_science_vacancies()

60
Готово, добавлено 60 вакансий
Ни одной вакансии не удалено
